In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from PIL import Image
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import urllib.request
from sklearn.preprocessing import LabelEncoder, StandardScaler
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import pandas as pd

# Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df_train = pd.read_csv("./data/train.csv", index_col = 0)
df_train.index.name = None
df_train

# Copia del dataframe original para transformaciones
df_train_processed = df_train.copy()

# 1) Mantener Company sin transformar (CatBoost maneja categóricas)

# 2) Transformar Product (primera palabra o última si es alfanumérico)
def extract_product_keyword(product):
    words = product.split()
    if words[-1].isalnum() and any(c.isalpha() for c in words[-1]) and any(c.isdigit() for c in words[-1]):
        return words[-1]  # Si es alfanumérico, tomar la última palabra
    return words[0]  # En otro caso, tomar la primera palabra

df_train_processed["Product"] = df_train_processed["Product"].apply(extract_product_keyword)

# 3) Modificar TypeName
# Función para categorizar TypeName
def categorize_type_name(type_name):
    type_name = type_name.lower()
    if "gaming" in type_name:
        return 3
    elif "workstation" in type_name:
        return 2
    else:
        return 1

# Aplicar la categorización a TypeName
df_train_processed["TypeName"] = df_train_processed["TypeName"].apply(categorize_type_name)

# 4) Transformar ScreenResolution tomando solo el último código alfanumérico
def extract_screen_res(resolution):
    matches = re.findall(r'\d+x\d+', resolution)
    return matches[-1] if matches else "Unknown"

df_train_processed["ScreenResolution"] = df_train_processed["ScreenResolution"].apply(extract_screen_res)

# Transformar ScreenResolution en dos columnas numéricas: Ancho y Alto
df_train_processed[['Screen_Width', 'Screen_Height']] = df_train_processed['ScreenResolution'].str.split('x', expand=True).astype(int)

# 5) Dividir CPU en tres columnas
def extract_cpu_parts(cpu):
    parts = cpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    third_word = parts[2] if len(parts) > 2 else "Unknown"
    last_word = parts[-1] if len(parts) > 0 else "Unknown"
    return first_word, third_word, last_word

df_train_processed[["CPU_Brand", "CPU_Model", "CPU_Speed"]] = df_train_processed["Cpu"].apply(lambda x: pd.Series(extract_cpu_parts(x)))

# Transformar CPU_Model en una variable numérica con sentido
def categorize_cpu(cpu):
    if "i7" in cpu:
        return 3
    elif "i5" in cpu:
        return 2
    elif "i3" in cpu:
        return 1
    else:
        return 1

df_train_processed["CPU_Model_number"] = df_train_processed["CPU_Model"].apply(categorize_cpu)

# 6) Dividir Memory en dos columnas
def extract_memory_parts(memory):
    parts = memory.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    last_word = parts[-1] if len(parts) > 1 else "Unknown"
    return first_word, last_word

df_train_processed[["Memory_Capacity", "Memory_Type"]] = df_train_processed["Memory"].apply(lambda x: pd.Series(extract_memory_parts(x)))

# 7) Dividir Gpu en tres columnas
def extract_gpu_parts(gpu):
    parts = gpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    second_word = parts[1] if len(parts) > 1 else "Unknown"
    last_word = parts[-1] if len(parts) > 2 else "Unknown"
    return first_word, second_word, last_word

df_train_processed[["GPU_Brand", "GPU_Type", "GPU_Model"]] = df_train_processed["Gpu"].apply(lambda x: pd.Series(extract_gpu_parts(x)))

# 8) Reducir OpSys a seis categorías
def categorize_os(os):
    os = os.lower()
    if "windows" in os:
        return 2
    elif "linux" in os:
        return 1
    elif "mac" in os or "macos" in os:
        return 3
    elif "android" in os:
        return 0
    elif "chrome" in os or "chromebook" in os:
        return 0
    else:
        return 0

df_train_processed["OpSys_type"] = df_train_processed["OpSys"].apply(categorize_os)
# Extraer el número del sistema operativo si existe
df_train_processed["OpSys_number"] = df_train_processed["OpSys"].str.extract(r'(\d+)')

# Convertir a número (NaN se llenará con 0 o algún otro valor si es necesario)
df_train_processed["OpSys_number"] = df_train_processed["OpSys_number"].fillna(0).astype(int)

# Limpiar y convertir columnas numéricas

# 1) Convertir Ram a numérico (eliminando "GB")
df_train_processed["Ram"] = df_train_processed["Ram"].str.replace("GB", "", regex=True).astype(int)

# 2) Convertir Weight a numérico (eliminando "kg")
df_train_processed["Weight"] = df_train_processed["Weight"].str.replace("kg", "", regex=True).astype(float)

# 3) Crear nuevas características
# Calcular el número total de píxeles
df_train_processed["Pixel_Count"] = df_train_processed["ScreenResolution"].apply(lambda x: int(x.split('x')[0]) * int(x.split('x')[1]) if 'x' in x else 0)

# Densidad de píxeles por pulgada (DPI)
df_train_processed["DPI"] = df_train_processed["Pixel_Count"] / (df_train_processed["Inches"] ** 2)

# Relación entre peso y tamaño de pantalla
df_train_processed["Weight_per_Inch"] = df_train_processed["Weight"] / df_train_processed["Inches"]

# Relación CPU y RAM
df_train_processed["CPU_Ratio"] = df_train_processed["CPU_Brand"].astype(str) + "_" + df_train_processed["Ram"].astype(str)

# Mantener las columnas categóricas como strings para CatBoost
df_train_processed["Company"] = df_train_processed["Company"].astype(str)
df_train_processed["Product"] = df_train_processed["Product"].astype(str)
df_train_processed["TypeName"] = df_train_processed["TypeName"].astype(str)
df_train_processed["CPU_Brand"] = df_train_processed["CPU_Brand"].astype(str)
df_train_processed["CPU_Model"] = df_train_processed["CPU_Model"].astype(str)
df_train_processed["CPU_Speed"] = df_train_processed["CPU_Speed"].str[:-3].astype(float)
df_train_processed["Memory_Capacity"] = df_train_processed["Memory_Capacity"].str.replace('TB', '000').str.replace('GB', '').astype(float)
df_train_processed["Memory_Type"] = df_train_processed["Memory_Type"].astype(str)
df_train_processed["GPU_Brand"] = df_train_processed["GPU_Brand"].astype(str)
df_train_processed["GPU_Model"] = df_train_processed["GPU_Model"].astype(str)
df_train_processed["OpSys"] = df_train_processed["OpSys"].astype(str)

# Eliminar columnas originales que ya fueron transformadas
df_train_processed.drop(columns=["OpSys", "Cpu", "Memory", "Gpu", "ScreenResolution"], inplace=True)

# Renombrar el dataset final
df_train_processed = df_train_processed

df_train_processed

df_test = pd.read_csv("./data/test.csv", index_col=0)
df_test.index.name = None
df_test

# Copia del dataframe original para transformaciones
df_test_processed = df_test.copy()

# 1) Mantener Company sin transformar (CatBoost maneja categóricas)

# 2) Transformar Product (primera palabra o última si es alfanumérico)
def extract_product_keyword(product):
    words = product.split()
    if words[-1].isalnum() and any(c.isalpha() for c in words[-1]) and any(c.isdigit() for c in words[-1]):
        return words[-1]  # Si es alfanumérico, tomar la última palabra
    return words[0]  # En otro caso, tomar la primera palabra

df_test_processed["Product"] = df_test_processed["Product"].apply(extract_product_keyword)

# 3) # Aplicar la categorización a TypeName
df_test_processed["TypeName"] = df_test_processed["TypeName"].apply(categorize_type_name)

# 4) Transformar ScreenResolution tomando solo el último código alfanumérico
def extract_screen_res(resolution):
    matches = re.findall(r'\d+x\d+', resolution)
    return matches[-1] if matches else "Unknown"

df_test_processed["ScreenResolution"] = df_test_processed["ScreenResolution"].apply(extract_screen_res)

# Transformar ScreenResolution en dos columnas numéricas: Ancho y Alto
df_test_processed[['Screen_Width', 'Screen_Height']] = df_test_processed['ScreenResolution'].str.split('x', expand=True).astype(int)

# 5) Dividir CPU en tres columnas
def extract_cpu_parts(cpu):
    parts = cpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    third_word = parts[2] if len(parts) > 2 else "Unknown"
    last_word = parts[-1] if len(parts) > 0 else "Unknown"
    return first_word, third_word, last_word

df_test_processed[["CPU_Brand", "CPU_Model", "CPU_Speed"]] = df_test_processed["Cpu"].apply(lambda x: pd.Series(extract_cpu_parts(x)))

# Transformar CPU_Model en una variable numérica con sentido
df_test_processed["CPU_Model_number"] = df_test_processed["CPU_Model"].apply(categorize_cpu)

# 6) Dividir Memory en dos columnas
def extract_memory_parts(memory):
    parts = memory.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    last_word = parts[-1] if len(parts) > 1 else "Unknown"
    return first_word, last_word

df_test_processed[["Memory_Capacity", "Memory_Type"]] = df_test_processed["Memory"].apply(lambda x: pd.Series(extract_memory_parts(x)))

# 7) Dividir Gpu en tres columnas
def extract_gpu_parts(gpu):
    parts = gpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    second_word = parts[1] if len(parts) > 1 else "Unknown"
    last_word = parts[-1] if len(parts) > 2 else "Unknown"
    return first_word, second_word, last_word
df_test_processed[["GPU_Brand", "GPU_Type", "GPU_Model"]] = df_test_processed["Gpu"].apply(lambda x: pd.Series(extract_gpu_parts(x)))

df_test_processed["OpSys_type"] = df_test_processed["OpSys"].apply(categorize_os)
# Extraer el número del sistema operativo si existe
df_test_processed["OpSys_number"] = df_test_processed["OpSys"].str.extract(r'(\d+)')

# Convertir a número (NaN se llenará con 0 o algún otro valor si es necesario)
df_test_processed["OpSys_number"] = df_test_processed["OpSys_number"].fillna(0).astype(int)

# Limpiar y convertir columnas numéricas

# 1) Convertir Ram a numérico (eliminando "GB")
df_test_processed["Ram"] = df_test_processed["Ram"].str.replace("GB", "", regex=True).astype(int)

# 2) Convertir Weight a numérico (eliminando "kg")
df_test_processed["Weight"] = df_test_processed["Weight"].str.replace("kg", "", regex=True).astype(float)

# 3) Crear nuevas características
# Calcular el número total de píxeles
df_test_processed["Pixel_Count"] = df_test_processed["ScreenResolution"].apply(lambda x: int(x.split('x')[0]) * int(x.split('x')[1]) if 'x' in x else 0)

# Densidad de píxeles por pulgada (DPI)
df_test_processed["DPI"] = df_test_processed["Pixel_Count"] / (df_test_processed["Inches"] ** 2)

# Relación entre peso y tamaño de pantalla
df_test_processed["Weight_per_Inch"] = df_test_processed["Weight"] / df_test_processed["Inches"]

# Relación CPU y RAM
df_test_processed["CPU_Ratio"] = df_test_processed["CPU_Brand"].astype(str) + "_" + df_test_processed["Ram"].astype(str)

# Mantener las columnas categóricas como strings para CatBoost
df_test_processed["Company"] = df_test_processed["Company"].astype(str)
df_test_processed["Product"] = df_test_processed["Product"].astype(str)
df_test_processed["TypeName"] = df_test_processed["TypeName"].astype(str)
df_test_processed["CPU_Brand"] = df_test_processed["CPU_Brand"].astype(str)
df_test_processed["CPU_Model"] = df_test_processed["CPU_Model"].astype(str)
df_test_processed["CPU_Speed"] = df_test_processed["CPU_Speed"].str[:-3].astype(float)
df_test_processed["Memory_Capacity"] = df_test_processed["Memory_Capacity"].str.replace('TB', '000').str.replace('GB', '').astype(float)
df_test_processed["Memory_Type"] = df_test_processed["Memory_Type"].astype(str)
df_test_processed["GPU_Brand"] = df_test_processed["GPU_Brand"].astype(str)
df_test_processed["GPU_Model"] = df_test_processed["GPU_Model"].astype(str)
df_test_processed["OpSys"] = df_test_processed["OpSys"].astype(str)

# Eliminar columnas originales que ya fueron transformadas
df_test_processed.drop(columns=["OpSys", "Cpu", "Memory", "Gpu", "ScreenResolution"], inplace=True)

# Renombrar el dataset final
df_test_processed = df_test_processed

df_test_processed

# Hacer una copia del dataset
df_encoded = df_train_processed.copy()

# Aplicar Label Encoding a todas las columnas categóricas
for col in df_encoded.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])

# Calcular la matriz de correlación con las variables categóricas transformadas
corr_matrix = df_encoded.corr()

# Definir un umbral de correlación mínima
correlation_threshold = 0.07

# Seleccionar variables con alta correlación con Price_in_euros
correlated_features = corr_matrix["Price_in_euros"].abs()
relevant_features = correlated_features[correlated_features > correlation_threshold].index.tolist()

# Filtrar el dataset con las variables más relevantes
df_train_filtered = df_encoded[relevant_features]

# Hacer una copia del dataset de test
df_test_encoded = df_test_processed.copy()

# Aplicar Label Encoding usando los mismos encoders de train
for col in df_test_encoded.select_dtypes(include=["object"]).columns:
    if col in df_train_processed.columns:  # Asegurar que la columna existe en train
        le = LabelEncoder()
        le.fit(df_train_processed[col])  # Usar las categorías aprendidas en train
        df_test_encoded[col] = df_test_encoded[col].map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

# Filtrar test con las mismas columnas seleccionadas en train
selected_features = df_train_filtered.drop(columns=["Price_in_euros"]).columns  # Excluir la variable objetivo

# Asegurar que el test solo tenga las columnas seleccionadas en train
df_test_filtered = df_test_encoded[selected_features]

# Relación entre RAM y CPU
df_train_filtered["Ram_per_CPU"] = df_train_filtered["Ram"] / (df_train_filtered["CPU_Speed"] + 0.1)
df_test_filtered["Ram_per_CPU"] = df_test_filtered["Ram"] / (df_test_filtered["CPU_Speed"] + 0.1)

# Tamaño de pantalla en relación con píxeles
df_train_filtered["Pixel_Density"] = df_train_filtered["Pixel_Count"] / (df_train_filtered["Inches"] ** 2)
df_test_filtered["Pixel_Density"] = df_test_filtered["Pixel_Count"] / (df_test_filtered["Inches"] ** 2)

# Eficiencia del almacenamiento (peso vs capacidad)
df_train_filtered["Storage_Efficiency"] = df_train_filtered["Memory_Capacity"] / (df_train_filtered["Weight"] + 0.1)
df_test_filtered["Storage_Efficiency"] = df_test_filtered["Memory_Capacity"] / (df_test_filtered["Weight"] + 0.1)

# Reordenar las columnas del dataframe según la correlación con "Price_in_euros"
target_correlation = df_train_filtered.corr()["Price_in_euros"].abs().sort_values(ascending=False)
ordered_columns = target_correlation.index.tolist()
df_train_filtered = df_train_filtered[ordered_columns]

# Función para reordenar el dataset de test en el mismo orden que el train
def reorder_columns(df_test, ordered_columns):
    # Seleccionar solo las columnas que están en el dataset de test
    common_columns = [col for col in ordered_columns if col in df_test.columns]
    return df_test[common_columns]

df_test_filtered = reorder_columns(df_test_filtered, ordered_columns)

# Seleccionar solo columnas numéricas excluyendo "Price_in_euros"
numerical_cols = df_train_processed.select_dtypes(include=["number"]).columns.tolist()
numerical_cols.remove("Price_in_euros")  # Excluir la variable objetivo

# Aplicar normalización con StandardScaler
scaler = StandardScaler()
df_train_processed[numerical_cols] = scaler.fit_transform(df_train_processed[numerical_cols])
df_test_processed[numerical_cols] = scaler.transform(df_test_processed[numerical_cols])  # Usar los mismos parámetros de train

print("Datos numéricos normalizados correctamente.")

# Separar variables predictoras y objetivo
X = df_train_filtered.drop(columns=["Price_in_euros"])  # Excluir variable objetivo
y = df_train_filtered["Price_in_euros"]

# Dividir en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar columnas categóricas (CatBoost maneja estas columnas directamente)
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

# Modelos a probar
models = {
    "Linear Regression": LinearRegression(),
    "CatBoost": CatBoostRegressor(loss_function="RMSE", verbose=0, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

# Optimización de hiperparámetros
param_grids = {
    "CatBoost": {"depth": [6, 8, 10], "learning_rate": [0.01, 0.05, 0.1], "iterations": [500, 1000, 1500]},
    "XGBoost": {"n_estimators": [50, 100, 200, 400], "learning_rate": [0.01, 0.05, 0.1, 0.2], "max_depth": [3, 5, 7, 9]},
    "Random Forest": {"n_estimators": [50, 100, 200], "max_depth": [None, 10, 20], "min_samples_split": [2, 5, 10]},
    "LightGBM": {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.05, 0.1], "max_depth": [3, 5, 7]}
}

best_models = {}

for name, model in models.items():
    if name in param_grids:  # Optimización solo para modelos con hiperparámetros
        search = RandomizedSearchCV(model, param_grids[name], n_iter=15, cv=3, scoring="neg_root_mean_squared_error", verbose=1, n_jobs=-1)
        search.fit(X_train, y_train)
        best_models[name] = search.best_estimator_
        print(f"{name} mejores hiperparámetros: {search.best_params_}")
    else:  # Para regresión lineal, entrenar directamente
        model.fit(X_train, y_train)
        best_models[name] = model

# Evaluar modelos optimizados
for name, model in best_models.items():
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"{name} RMSE: {rmse}")

# Seleccionar el mejor modelo basado en RMSE
best_model_name = min(best_models, key=lambda k: np.sqrt(mean_squared_error(y_val, best_models[k].predict(X_val))))
best_model = best_models[best_model_name]
print(f"Mejor modelo seleccionado: {best_model_name}")

# Generar predicciones para Kaggle
X_test_filtered = df_test_filtered  # Usar test filtrado con las mismas features

y_test_pred = best_model.predict(X_test_filtered)

# Guardar las predicciones en un archivo CSV para Kaggle
df_submission = pd.DataFrame({"laptop_ID": df_test.index, "Price_in_euros": y_test_pred})
df_submission.to_csv("submission_best_model_2.csv", index=False)

print(f"Predicciones guardadas en 'submission_best_model.csv' usando {best_model_name}.")



Datos numéricos normalizados correctamente.
Fitting 3 folds for each of 15 candidates, totalling 45 fits
CatBoost mejores hiperparámetros: {'learning_rate': 0.05, 'iterations': 1000, 'depth': 6}
Fitting 3 folds for each of 15 candidates, totalling 45 fits
XGBoost mejores hiperparámetros: {'n_estimators': 400, 'max_depth': 3, 'learning_rate': 0.1}
Fitting 3 folds for each of 15 candidates, totalling 45 fits
Random Forest mejores hiperparámetros: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': None}
Fitting 3 folds for each of 15 candidates, totalling 45 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000167 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 816
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 26
[LightGBM] [Info] Start training from score 1103.789314
[LightGBM] [Warning